# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = ("../output_data/cities.csv")
cities =pd.read_csv(file)
cities.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
Locations= cities[["Lat", "Lng"]]
Locations.head()

Humidity= cities["Humidity"].astype(float)
Humidity.head()

0    72.0
1    74.0
2    73.0
3    39.0
4    76.0
Name: Humidity, dtype: float64

In [4]:
#Add Heatmap layer to map
fig = gmaps.figure(zoom_level =2, center= (33.7, 84.4))

# Create heat layer
heat_layer = gmaps.heatmap_layer(Locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions
temp = cities[cities['Max Temp']> 70 ]
temps = temp[temp['Max Temp']<80] 
windSpeed =  temps[temps['Wind Speed'] < 10]
cloudiness_df =  windSpeed[windSpeed['Cloudiness'] == 0]
cloudiness_df_df = cloudiness_df.reset_index()
del cloudiness_df_df ["index"]
cloudiness_df_df 


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create hotel_df with hotel name column
hotel_df = pd.DataFrame(cloudiness_df_df, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,opuwo,NaN,-18.06,13.84,72.06,NaN
1,ponta do sol,BR,-20.63,-46.00,73.68,NaN
2,cidreira,BR,-30.17,-50.22,76.56,NaN
3,morondava,MG,-20.30,44.28,79.08,NaN
4,vaini,IN,15.34,74.49,78.90,NaN
5,nador,MA,35.17,-2.93,75.20,NaN
6,mogok,MM,22.92,96.51,72.24,NaN
7,birjand,IR,32.86,59.22,71.60,NaN


In [7]:

# geocoordinates
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
idx = 0  
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {"location": f"{lat},{lng}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
hotel_response = response.json()
results = hotel_response['results']
if len(results) > 0 :
        name =hotel_response['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
        idx = idx + 1


hotel_df


,City,Country,Lat,Lng,Max Temp,Hotel Name
0,opuwo,NaN,-18.06,13.84,72.06,هتل پرويز
1,ponta do sol,BR,-20.63,-46.00,73.68,NaN
2,cidreira,BR,-30.17,-50.22,76.56,NaN
3,morondava,MG,-20.30,44.28,79.08,NaN
4,vaini,IN,15.34,74.49,78.90,NaN
5,nador,MA,35.17,-2.93,75.20,NaN
6,mogok,MM,22.92,96.51,72.24,NaN
7,birjand,IR,32.86,59.22,71.60,NaN


In [8]:
pprint(hotel_response)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 32.8802023, 'lng': 59.216279},
                           'viewport': {'northeast': {'lat': 32.88151937989272,
                                                      'lng': 59.21771057989272},
                                        'southwest': {'lat': 32.87881972010727,
                                                      'lng': 59.21501092010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'هتل پرويز',
              'place_id': 'ChIJ8dNk011nGj8RZI51wiSCBsY',
              'rating': 3,
              'reference': 'ChIJ8dNk011nGj8RZI51wiSCBsY',
              'scope': 'GOOGLE',
              'types': ['lodging', 'point_of_interest', 'es

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))